# Imports and preparation

In [1]:
import os

In [2]:
import numpy as np

In [3]:
import pandas as pd

In [4]:
from tqdm import tqdm

In [5]:
import time, gc

In [6]:
import importlib

In [7]:
from itertools import product


In [8]:
from util import util_elastic

OpenAI tiktoken module is not available for Python < 3.8,Linux ARM64 and AARCH64. Falling back to GPT2TokenizerFast.


In [9]:
from util import util_pipeline

In [10]:
from util import util_experiment_juris_tcu as util_experiment

In [11]:
# Para ter repetibilidade nos resultados
random_state = 1

# Tratar valores infinitos (+ e -) como np.NaN
pd.options.mode.use_inf_as_na = True

# IMPORTANTE para tornar figuras interativas
# %matplotlib notebook

# Tamanho padrão das figuras
figsize=(10,6)

pd.set_option('display.max_row', 1000)

pd.set_option('display.max_columns', 50)

pd.set_option('display.column_space', 100)
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.width', 200)


In [12]:
DATASET_NAME = 'juris_tcu'

In [13]:
PATH_QUERY = f'../data/{DATASET_NAME}/query.csv'
PATH_QREL =  f'../data/{DATASET_NAME}/qrel.csv'

# Data load

## Query data load

In [14]:
df_query = pd.read_csv(PATH_QUERY)

In [15]:
df_query.shape

(150, 3)

In [16]:
df_query.head()

,ID,TEXT,SOURCE
0,1,técnica e preço,search log
1,2,restos a pagar,search log
2,3,aditivo a contrato,search log
3,4,adesão a ata de registro de preços,search log
4,5,sobrepreço e superfaturamento,search log


In [17]:
df_query = df_query.reset_index(drop=True)

# Expand query

## Create reference to index (juris_tcu_index)

In [18]:
INDEX_NAME = 'indir_juris_tcu_index'

In [19]:
index = util_elastic.return_index(parm_index_name=INDEX_NAME)


Qtd de documentos 13255

Qtd de embeddings 13255

Documento.id=5: <Document: id=5, content='O termo é "Abono de permanência em serviço".
Abono de permanência em serviço tem definição: "O abono...'>


## Create pipeline

In [20]:
# Test query
# parm_query = "trata-se de uma denúncia contra o prefeito de Manhuaçu por não haver pago os funcionários da área de limpeza urbana"
parm_query = "despesa sem cobertura contratual"

In [40]:
pipes = []

In [41]:
ranker_type = 'PTT5_INDIR_400' #'PTT5_BASE' #'PTT5_INDIR_83' # 'MT5_3B' # 'MT5_TRAINED_LIM50_800' # 'MINILM_TRAINED_19000'4

In [42]:
FILTER_TERMO = {'class':['Termo']}

In [24]:
raise Exception('Para aqui')

Exception: Para aqui

### bm25

In [ ]:
del pipe_bm25_ranker

In [ ]:
%%time
pipe_bm25_ranker = util_pipeline.return_pipeline_bm25_reranker(index, ranker_type, parm_limit_query_size=50)

Loading PTT5_INDIR_106 with limit_query_size=50
CPU times: user 10.1 s, sys: 1.51 s, total: 11.6 s
Wall time: 8.57 s


In [ ]:
pipes.append({'PIPE_NAME': 'pipe_bm25_ranker',
              'PIPE_OBJECT': pipe_bm25_ranker,
              'RETRIEVER_TYPE': 'bm25',  
              'RETRIEVER_MODEL_NAME': '', 
              'RANKER_MODEL_NAME': util_pipeline.return_ranker_name(ranker_type)})

In [ ]:
doctos_retornados_ranker = pipe_bm25_ranker.run(query=parm_query, params= {"Retriever":{"top_k":100, "filters": {"count_index_total": {"$gte": 5} }}})
util_pipeline.detail_document_found(doctos_retornados_ranker)

Parâmetros usados: {'Retriever': {'top_k': 100, 'filters': {'count_index_total': {'$gte': 5}}}}
Consulta: despesa sem cobertura contratual
Qtd documentos retornados: 100
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Garantia contratual', '685', -0.010465267114341259]
1 : ['Contrato administrativo', '331', -0.011969292536377907]
2 : ['Dispensa de licitação', '459', -0.06604985892772675]
3 : ['Contratação direta', '327', -0.14308224618434906]
4 : ['Contrato', '330', -0.2242659628391266]
5 : ['Despesa', '421', -0.35243895649909973]
6 : ['Prorrogação de contrato', '1292', -0.45849138498306274]
7 : ['Pessoal', '1131', -0.5640890598297119]
8 : ['Plano de saúde', '1145', -0.5767648220062256]
9 : ['Liquidação da despesa', '950', -0.901408851146698]
10 : ['Garantia da proposta', '2534', -0.9434277415275574]


In [ ]:
doctos_retornados_ranker = pipe_bm25_ranker.run(query=parm_query, params= {"Retriever":{"top_k":30, "filters": {"count_index_total": {"$gte": 5} }}})
util_pipeline.detail_document_found(doctos_retornados_ranker)

Parâmetros usados: {'Retriever': {'top_k': 30, 'filters': {'count_index_total': {'$gte': 5}}}}
Consulta: despesa sem cobertura contratual
Qtd documentos retornados: 30
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Garantia contratual', '685', -0.02296389825642109]
1 : ['Contrato', '330', -0.11135894805192947]
2 : ['Inexecução', '800', -0.18585744500160217]
3 : ['Despesa', '421', -0.38263654708862305]
4 : ['Débito', '370', -0.575980007648468]
5 : ['Contratação direta', '327', -1.0081050395965576]
6 : ['Liquidação da despesa', '950', -1.2624561786651611]
7 : ['Ordenação de despesa', '14780', -1.2977583408355713]
8 : ['Despesa pública', '691', -1.6750495433807373]
9 : ['Despesa de custeio', '2282', -2.073354959487915]
10 : ['Custeio', '1102', -2.1612818241119385]


In [ ]:
doctos_retornados_ranker = pipe_bm25_ranker.run(query=parm_query, params= {"Retriever":{"top_k":30}})
util_pipeline.detail_document_found(doctos_retornados_ranker)

Parâmetros usados: {'Retriever': {'top_k': 30}}
Consulta: despesa sem cobertura contratual
Qtd documentos retornados: 30
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Garantia contratual', '685', -0.022963782772421837]
1 : ['Gestão contratual', '1104190', -0.2078995257616043]
2 : ['Despesa sem prévio empenho', '1104589', -0.31134384870529175]
3 : ['Despesa', '421', -0.38272443413734436]
4 : ['Carta contrato', '1929', -0.9164692759513855]
5 : ['Reconhecimento de dívida', '1105310', -1.077577829360962]
6 : ['Despesa especial', '1104118', -1.1674960851669312]
7 : ['Cobertura (Obra pública)', '5093', -1.1881086826324463]
8 : ['Liquidação da despesa', '950', -1.2624561786651611]
9 : ['Ordenação de despesa', '14780', -1.296999216079712]
10 : ['Despesa pública', '691', -1.6742503643035889]


In [ ]:
doctos_retornados_ranker = pipe_bm25_ranker.run(query=parm_query, params= {"Retriever":{"top_k":50, "filters": FILTER_TERMO}})
util_pipeline.detail_document_found(doctos_retornados_ranker)

Parâmetros usados: {'Retriever': {'top_k': 50, 'filters': {'class': ['Termo']}}}
Consulta: despesa sem cobertura contratual
Qtd documentos retornados: 50
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Garantia contratual', '685', -0.022906577214598656]
1 : ['Inexecução', '800', -0.1862320601940155]
2 : ['Gestão contratual', '1104190', -0.20773528516292572]
3 : ['Negativa de atendimento', '2196', -0.2135389894247055]
4 : ['Despesa sem prévio empenho', '1104589', -0.31134384870529175]
5 : ['Despesa', '421', -0.38272443413734436]
6 : ['Débito', '370', -0.5758218765258789]
7 : ['Carta contrato', '1929', -0.9164692759513855]
8 : ['Contratação direta', '327', -1.0090245008468628]
9 : ['Reconhecimento de dívida', '1105310', -1.077577829360962]
10 : ['Despesa especial', '1104118', -1.168460726737976]


### join

In [24]:
importlib.reload(util_pipeline)

<module 'util.util_pipeline' from '/home/borela/fontes/ind-ir/code/util/util_pipeline.py'>

In [43]:
FILTER_TERMO, ranker_type

({'class': ['Termo']}, 'PTT5_INDIR_400')

In [44]:
%%time
pipe_join_bm25_sts_reranker = util_pipeline.return_pipeline_join_bm25_sts_reranker(index, ranker_type, parm_limit_query_size=50)

CPU times: user 2.95 s, sys: 304 ms, total: 3.26 s
Wall time: 1.72 s


In [45]:
pipes.append({'PIPE_NAME': 'pipe_join_bm25_sts_reranker',
              'PIPE_OBJECT': pipe_join_bm25_sts_reranker,
              'RETRIEVER_TYPE': 'join_sts_bm25',  
              'RETRIEVER_MODEL_NAME': '', 
              'RANKER_MODEL_NAME': util_pipeline.return_ranker_name(ranker_type)})

In [48]:
%%time
doctos_retornados_ranker = pipe_join_bm25_sts_reranker.run(query=parm_query, 
 params={"Bm25Retriever": {"top_k": 30}, "StsRetriever": {"top_k": 30}, "Ranker": {"top_k": 30}, "filters": FILTER_TERMO})
util_pipeline.detail_document_found(doctos_retornados_ranker, 30)

Parâmetros usados: {'Bm25Retriever': {'top_k': 30}, 'StsRetriever': {'top_k': 30}, 'Ranker': {'top_k': 30}, 'filters': {'class': ['Termo']}}
Consulta: despesa sem cobertura contratual
Qtd documentos retornados: 30
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Garantia contratual', '685', -0.0038250156212598085]
1 : ['Custo não renovável', '1104580', -0.05328916013240814]
2 : ['Carta contrato', '1929', -0.20729772746562958]
3 : ['Regime de execução contratual', '14904', -0.31326600909233093]
4 : ['Despesa', '421', -0.5544226169586182]
5 : ['Gestão contratual', '1104190', -0.9828687906265259]
6 : ['Pagamento indevido', '2041', -1.6879194974899292]
7 : ['Contrapartida', '15152', -1.7221322059631348]
8 : ['Reconhecimento de dívida', '1105310', -1.755555272102356]
9 : ['Rescisão contratual', '1392', -1.7752190828323364]
10 : ['Plano de saúde', '1145', -1.891367793083191]
11 : ['Despesa pública', '691', -1.9357742071151733]
12 : ['Despesa sem prévio empenho', '1104589', -2.1

In [32]:
%%time
doctos_retornados_ranker = pipe_join_bm25_sts_reranker.run(query=parm_query, 
 params={"Bm25Retriever": {"top_k": 300}, "StsRetriever": {"top_k": 300}, "Ranker": {"top_k": 30}, "filters": FILTER_TERMO})
util_pipeline.detail_document_found(doctos_retornados_ranker, 30)

Parâmetros usados: {'Bm25Retriever': {'top_k': 300}, 'StsRetriever': {'top_k': 300}, 'Ranker': {'top_k': 30}, 'filters': {'class': ['Termo']}}
Consulta: despesa sem cobertura contratual
Qtd documentos retornados: 30
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Contrato verbal', '332', -0.003175456775352359]
1 : ['Garantia contratual', '685', -0.0038250156212598085]
2 : ['Contrato administrativo', '331', -0.006363249849528074]
3 : ['Custo não renovável', '1104580', -0.05328916013240814]
4 : ['Termo de contrato', '1981', -0.17668592929840088]
5 : ['Execução de contrato', '2494', -0.1847897619009018]
6 : ['Carta contrato', '1929', -0.20805832743644714]
7 : ['Regime de execução contratual', '14904', -0.31265631318092346]
8 : ['Objeto do contrato', '16071', -0.37492939829826355]
9 : ['Prorrogação de contrato', '1292', -0.47048693895339966]
10 : ['Despesa', '421', -0.5533663034439087]
11 : ['Passivo trabalhista', '15319', -0.6800861954689026]
12 : ['Contrato', '330', -0.696

In [ ]:
%%time
doctos_retornados_ranker = pipe_join_bm25_sts_reranker.run(query=parm_query, 
 params={"Bm25Retriever": {"top_k": 100}, "StsRetriever": {"top_k": 100}, "Ranker": {"top_k": 20}, "filters": FILTER_TERMO})
util_pipeline.detail_document_found(doctos_retornados_ranker)

Parâmetros usados: {'Bm25Retriever': {'top_k': 100}, 'StsRetriever': {'top_k': 100}, 'Ranker': {'top_k': 20}, 'filters': {'class': ['Termo']}}
Consulta: despesa sem cobertura contratual
Qtd documentos retornados: 20
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Garantia contratual', '685', -0.010465267114341259]
1 : ['Custo não renovável', '1104580', -0.02873079478740692]
2 : ['Dispensa de licitação', '459', -0.06621533632278442]
3 : ['Gestão contratual', '1104190', -0.07890946418046951]
4 : ['Negativa de atendimento', '2196', -0.12835638225078583]
5 : ['Contratação direta', '327', -0.14302872121334076]
6 : ['Contrato', '330', -0.2242659628391266]
7 : ['Despesa', '421', -0.35243895649909973]
8 : ['Despesa sem prévio empenho', '1104589', -0.4060640037059784]
9 : ['Carta contrato', '1929', -0.42900973558425903]
10 : ['Plano de saúde', '1145', -0.5769578814506531]
CPU times: user 2.71 s, sys: 6.76 ms, total: 2.72 s
Wall time: 2.82 s


In [ ]:
%%time
doctos_retornados_ranker = pipe_join_bm25_sts_reranker.run(query=parm_query, 
 params={"Bm25Retriever": {"top_k": 50}, "StsRetriever": {"top_k": 100}, "Ranker": {"top_k": 30}, "filters": FILTER_TERMO})
util_pipeline.detail_document_found(doctos_retornados_ranker, 30)

Parâmetros usados: {'Bm25Retriever': {'top_k': 50}, 'StsRetriever': {'top_k': 100}, 'Ranker': {'top_k': 30}, 'filters': {'class': ['Termo']}}
Consulta: despesa sem cobertura contratual
Qtd documentos retornados: 30
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Garantia contratual', '685', -0.02296389825642109]
1 : ['Custo não renovável', '1104580', -0.04771995544433594]
2 : ['Inexecução', '800', -0.18585774302482605]
3 : ['Gestão contratual', '1104190', -0.20773528516292572]
4 : ['Negativa de atendimento', '2196', -0.21367570757865906]
5 : ['Despesa sem prévio empenho', '1104589', -0.31156131625175476]
6 : ['Despesa', '421', -0.38263654708862305]
7 : ['Débito', '370', -0.5751016139984131]
8 : ['Plano de saúde', '1145', -0.7520239353179932]
9 : ['Contrapartida', '15152', -0.846224844455719]
10 : ['Carta contrato', '1929', -0.9168330430984497]
11 : ['Contratação direta', '327', -1.007680892944336]
12 : ['Reconhecimento de dívida', '1105310', -1.0774281024932861]
13 : ['P

In [ ]:
%%time
doctos_retornados_ranker = pipe_join_bm25_sts_reranker.run(query=parm_query, 
 params={"Bm25Retriever": {"top_k": 100}, "StsRetriever": {"top_k": 50}, "Ranker": {"top_k": 30}, "filters": FILTER_TERMO})
util_pipeline.detail_document_found(doctos_retornados_ranker, 30)

Parâmetros usados: {'Bm25Retriever': {'top_k': 100}, 'StsRetriever': {'top_k': 50}, 'Ranker': {'top_k': 30}, 'filters': {'class': ['Termo']}}
Consulta: despesa sem cobertura contratual
Qtd documentos retornados: 30
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Garantia contratual', '685', -0.02296389825642109]
1 : ['Custo não renovável', '1104580', -0.04771995544433594]
2 : ['Contrato', '330', -0.11136246472597122]
3 : ['Inexecução', '800', -0.18585774302482605]
4 : ['Gestão contratual', '1104190', -0.20773528516292572]
5 : ['Negativa de atendimento', '2196', -0.21367570757865906]
6 : ['Despesa sem prévio empenho', '1104589', -0.31156131625175476]
7 : ['Despesa', '421', -0.38263654708862305]
8 : ['Débito', '370', -0.5751006603240967]
9 : ['Dispensa de licitação', '459', -0.6776613593101501]
10 : ['Contratado', '1933', -0.7043430805206299]
11 : ['Plano de saúde', '1145', -0.7520239353179932]
12 : ['Contrapartida', '15152', -0.846224844455719]
13 : ['Carta contrato', '19

In [ ]:
%%time
doctos_retornados_ranker = pipe_join_bm25_sts_reranker.run(query=parm_query, 
 params={"Bm25Retriever": {"top_k": 50}, "StsRetriever": {"top_k": 50}, "Ranker": {"top_k": 30}, "filters": FILTER_TERMO})
util_pipeline.detail_document_found(doctos_retornados_ranker, 30)

Parâmetros usados: {'Bm25Retriever': {'top_k': 50}, 'StsRetriever': {'top_k': 50}, 'Ranker': {'top_k': 30}, 'filters': {'class': ['Termo']}}
Consulta: despesa sem cobertura contratual
Qtd documentos retornados: 30
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Custo não renovável', '1104580', -0.3954627215862274]
1 : ['Despesa sem prévio empenho', '1104589', -0.47731465101242065]
2 : ['Negativa de atendimento', '2196', -4.204003810882568]
3 : ['Evento sem ônus', '1104010', -5.777669429779053]
4 : ['Reconhecimento de dívida', '1105310', -5.884185314178467]
5 : ['Contratação direta', '327', -7.314028739929199]
6 : ['Plano de saúde', '1145', -8.249095916748047]
7 : ['Empenho', '497', -8.447331428527832]
8 : ['Empenho prévio', '1104618', -9.021744728088379]
9 : ['Juros compensatórios', '1101030', -9.985469818115234]
10 : ['Sub-rogação', '1487', -10.005921363830566]
11 : ['Cobertura (Saúde)', '2181', -10.110730171203613]
12 : ['Passivo compensado', '4488', -10.19416332244873

In [ ]:
%%time
doc_return = pipe_join_bm25_sts_reranker.run(query=parm_query, 
 params={"Bm25Retriever": {"top_k": 30}, "StsRetriever": {"top_k": 30}, "Ranker": {"top_k": 30}, "filters": FILTER_TERMO})
util_pipeline.detail_document_found(doc_return, 30)

Parâmetros usados: {'Bm25Retriever': {'top_k': 30}, 'StsRetriever': {'top_k': 30}, 'Ranker': {'top_k': 30}, 'filters': {'class': ['Termo']}}
Consulta: despesa sem cobertura contratual
Qtd documentos retornados: 30
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Garantia contratual', '685', -0.02296389825642109]
1 : ['Custo não renovável', '1104580', -0.04771995544433594]
2 : ['Gestão contratual', '1104190', -0.20767620205879211]
3 : ['Despesa sem prévio empenho', '1104589', -0.31156131625175476]
4 : ['Despesa', '421', -0.3824957013130188]
5 : ['Plano de saúde', '1145', -0.7520239353179932]
6 : ['Contrapartida', '15152', -0.846224844455719]
7 : ['Carta contrato', '1929', -0.9168330430984497]
8 : ['Reconhecimento de dívida', '1105310', -1.078168272972107]
9 : ['Passivo financeiro', '4506', -1.160717248916626]
10 : ['Despesa especial', '1104118', -1.1673153638839722]
11 : ['Cobertura (Obra pública)', '5093', -1.1901191473007202]
12 : ['Liquidação da despesa', '950', -1.2624

In [ ]:
for docto in doc_return['documents'][:5]:
    print(docto.meta['name'])

Garantia contratual
Custo não renovável
Gestão contratual
Despesa sem prévio empenho
Despesa


## Create expansion

In [50]:
from unidecode import unidecode
import string
import re


In [51]:
def set_normalize_text(text):
    # Remover espaços duplicados e espaços extras à direita e à esquerda
    text = re.sub(r'\s+', ' ', text.lower()).strip()
    # Remover acentos e caracteres especiais
    text = unidecode(text)
    # Separar as palavras em substrings
    words = text.split()
    # Retornar um conjunto de palavras normalizadas
    return set(words)

In [52]:
def test_similar_or_subtext(text1, text2):
    normalized_text1 = set_normalize_text(text1)
    normalized_text2 = set_normalize_text(text2)
    return normalized_text1 == normalized_text2 or normalized_text1.issubset(normalized_text2)


In [53]:
set_normalize_text('olá comO   vai voc? está bem.')

{'bem.', 'como', 'esta', 'ola', 'vai', 'voc?'}

In [54]:
test_similar_or_subtext("Licitação de técnica e preço","técnica e preço")

False

In [55]:
def expand_query(parm_pipe, parm_df_query, num_expansion_max):
    if num_expansion_max > 30:
        raise Exception('Review limit in code for num_expansion_max of 30!')
    count_expansion = [inteiro + 1 for inteiro in range(num_expansion_max)]
    for index, row in tqdm(parm_df_query.iterrows()):
        query_id = row['ID']
        doc_return = parm_pipe.run(query=row['TEXT'], params={"Bm25Retriever": {"top_k": 30}, "StsRetriever": {"top_k": 30}, "Ranker": {"top_k": num_expansion_max}, "filters": FILTER_TERMO})

        termos = []
        for docto in doc_return['documents'][:num_expansion_max * 2]:
            if not test_similar_or_subtext(docto.meta['name'], row['TEXT']): # and docto.meta['name'] not in termos:
                termos.append(docto.meta['name'])
            if len(termos) == num_expansion_max:
                break
        # print(len(termos), termos)

        for count in count_expansion:
            col_name = f"{count}"
            termos_concatenados = ', '.join(termos[:count]).rstrip(', ')
            parm_df_query.loc[index, col_name] = f"{row['TEXT']} - {termos_concatenados}"


In [56]:
df_query = pd.read_csv(PATH_QUERY)

In [57]:
%%time
expand_query(parm_pipe = pipe_join_bm25_sts_reranker, parm_df_query=df_query, num_expansion_max=10)

150it [01:01,  2.42it/s]

CPU times: user 52.4 s, sys: 49.7 ms, total: 52.4 s
Wall time: 1min 1s


In [58]:
import torch

In [59]:
torch.cuda.empty_cache()

In [60]:
# ptt5-indir-400
df_query.head()

,ID,TEXT,SOURCE,1,2,3,4,5,6,7,8,9,10
0,1,técnica e preço,search log,técnica e preço - Licitação de técnica e preço,"técnica e preço - Licitação de técnica e preço, Proposta técnica","técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço de mercado","técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço de mercado, Proposta de preço","técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço de mercado, Proposta de preço, Preço unitário","técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço de mercado, Proposta de preço, Preço unitário, Proposta","técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço de mercado, Proposta de preço, Preço unitário, Proposta, Tipo de licitação","técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço de mercado, Proposta de preço, Preço unitário, Proposta, Tipo de licitação, Preço global","técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço de mercado, Proposta de preço, Preço unitário, Proposta, Tipo de licitação, Preço global, Gestão tecnológica","técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço de mercado, Proposta de preço, Preço unitário, Proposta, Tipo de licitação, Preço global, Gestão tecnológica"
1,2,restos a pagar,search log,restos a pagar - Recolhimento,"restos a pagar - Recolhimento, Quitação","restos a pagar - Recolhimento, Quitação, Cota-parte","restos a pagar - Recolhimento, Quitação, Cota-parte, Passivo","restos a pagar - Recolhimento, Quitação, Cota-parte, Passivo, Crime contra as finanças públicas","restos a pagar - Recolhimento, Quitação, Cota-parte, Passivo, Crime contra as finanças públicas, Liquidação judicial","restos a pagar - Recolhimento, Quitação, Cota-parte, Passivo, Crime contra as finanças públicas, Liquidação judicial, Passivo trabalhista","restos a pagar - Recolhimento, Quitação, Cota-parte, Passivo, Crime contra as finanças públicas, Liquidação judicial, Passivo trabalhista, Passivo compensado","restos a pagar - Recolhimento, Quitação, Cota-parte, Passivo, Crime contra as finanças públicas, Liquidação judicial, Passivo trabalhista, Passivo compensado, Inadimplência","restos a pagar - Recolhimento, Quitação, Cota-parte, Passivo, Crime contra as finanças públicas, Liquidação judicial, Passivo trabalhista, Passivo compensado, Inadimplência"
2,3,aditivo a contrato,search log,aditivo a contrato - Termo aditivo,"aditivo a contrato - Termo aditivo, Contrato administrativo","aditivo a contrato - Termo aditivo, Contrato administrativo, Reformulação","aditivo a contrato - Termo aditivo, Contrato administrativo, Reformulação, Acréscimo","aditivo a contrato - Termo aditivo, Contrato administrativo, Reformulação, Acréscimo, Reprovabilidade","aditivo a contrato - Termo aditivo, Contrato administrativo, Reformulação, Acréscimo, Reprovabilidade, Atualização","aditivo a contrato - Termo aditivo, Contrato administrativo, Reformulação, Acréscimo, Reprovabilidade, Atualização, Formalização do contrato","aditivo a contrato - Termo aditivo, Contrato administrativo, Reformulação, Acréscimo, Reprovabilidade, Atualização, Formalização do contrato, Aditivo quantitativo","aditivo a contrato - Termo aditivo, Contrato administrativo, Reformulação, Acréscimo, Reprovabilidade, Atualização, Formalização do contrato, Aditivo quantitativo","aditivo a contrato - Termo aditivo, Contrato administrativo, Reformulação, Acréscimo, Reprovabilidade, Atualização, Formalização do contrato, Aditivo quantitativo"
3,4,adesão a ata de registro de preços,search log,adesão a ata de registro de preços - Sistema de Registro de Preços,"adesão a ata de registro de preços - Sistema de Registro de Preços, Contrato de adesão","adesão a ata de registro de preços - Sistema de Registro de Preços, Contrato de adesão, Impugnação","adesão a ata de registro de preços - Sistema de Registro de Preços, Contrato de adesão, Impugnação, Aval","adesão a ata de registro de preços - Sistema de

In [ ]:
df_query.head()

,ID,TEXT,SOURCE,1,2,3,4,5,6,7,8,9,10
0,1,técnica e preço,search log,técnica e preço - Licitação de técnica e preço,"técnica e preço - Licitação de técnica e preço, Proposta técnica","técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço de mercado","técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço de mercado, Proposta de preço","técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço de mercado, Proposta de preço, Preço unitário","técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço de mercado, Proposta de preço, Preço unitário, Proposta","técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço de mercado, Proposta de preço, Preço unitário, Proposta, Bens e serviços de informática","técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço de mercado, Proposta de preço, Preço unitário, Proposta, Bens e serviços de informática, Qualificação técnica","técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço de mercado, Proposta de preço, Preço unitário, Proposta, Bens e serviços de informática, Qualificação técnica, Documento téc...","técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço de mercado, Proposta de preço, Preço unitário, Proposta, Bens e serviços de informática, Qualificação técnica, Documento téc..."
1,2,restos a pagar,search log,restos a pagar - Recolhimento,"restos a pagar - Recolhimento, Exercício financeiro anterior","restos a pagar - Recolhimento, Exercício financeiro anterior, Passivo circulante","restos a pagar - Recolhimento, Exercício financeiro anterior, Passivo circulante, Juros","restos a pagar - Recolhimento, Exercício financeiro anterior, Passivo circulante, Juros, Cota-parte","restos a pagar - Recolhimento, Exercício financeiro anterior, Passivo circulante, Juros, Cota-parte, Despesas de exercícios anteriores","restos a pagar - Recolhimento, Exercício financeiro anterior, Passivo circulante, Juros, Cota-parte, Despesas de exercícios anteriores, Obrigação","restos a pagar - Recolhimento, Exercício financeiro anterior, Passivo circulante, Juros, Cota-parte, Despesas de exercícios anteriores, Obrigação, Crime contra as finanças públicas","restos a pagar - Recolhimento, Exercício financeiro anterior, Passivo circulante, Juros, Cota-parte, Despesas de exercícios anteriores, Obrigação, Crime contra as finanças públicas, Quitação","restos a pagar - Recolhimento, Exercício financeiro anterior, Passivo circulante, Juros, Cota-parte, Despesas de exercícios anteriores, Obrigação, Crime contra as finanças públicas, Quitação"
2,3,aditivo a contrato,search log,aditivo a contrato - Termo aditivo,"aditivo a contrato - Termo aditivo, Aditivo qualitativo","aditivo a contrato - Termo aditivo, Aditivo qualitativo, Acréscimo","aditivo a contrato - Termo aditivo, Aditivo qualitativo, Acréscimo, Contrato administrativo","aditivo a contrato - Termo aditivo, Aditivo qualitativo, Acréscimo, Contrato administrativo, Aditivo quantitativo","aditivo a contrato - Termo aditivo, Aditivo qualitativo, Acréscimo, Contrato administrativo, Aditivo quantitativo, Formalização do contrato","aditivo a contrato - Termo aditivo, Aditivo qualitativo, Acréscimo, Contrato administrativo, Aditivo quantitativo, Formalização do contrato, Reformulação","aditivo a contrato - Termo aditivo, Aditivo qualitativo, Acréscimo, Contrato administrativo, Aditivo quantitativo, Formalização do contrato, Reformulação, Atualização","aditivo a contrato - Termo aditivo, Aditivo qualitativo, Acréscimo, Contrato administrativo, Aditivo quantitativo, Formalização do contrato, Reformulação, Atualização","aditivo a contrato - Termo aditivo, Aditivo qualitativo, Acréscimo, Contrato administrativo, Aditivo quantitativo, Formalização do contrato, Reformulação, Atualização"
3,4,adesão a ata de registro de preços,search log,adesão a ata de registro de preços - Sistema de Registro de Preços,"adesão a ata de registro de preços - Sistema de 

In [61]:
name_expansor = 'join_30_ptt5_indir_400'

In [62]:
path_expansion = f'../data/juris_tcu_expanded_query/query_10_expansion_{name_expansor}.csv'

In [63]:
df_query.to_csv(path_expansion, index=False)

# Exploring expansion

In [ ]:
EXPANSOR_CRITERIA = 'join_30_ptt5_indir_106' # 'join_30_ptt5_base' # 'join_30_ptt5_indir_106' #  'join_30_ptt5_base' #'join_30_ptt5_indir'

In [ ]:
PATH_QUERY =  f'../data/juris_tcu_expanded_query/query_10_expansion_{EXPANSOR_CRITERIA}.csv' 

In [ ]:
df_query = pd.read_csv(PATH_QUERY)

In [ ]:
df_query.head()

,ID,TEXT,SOURCE,1,2,3,4,5,6,7,8,9,10
0,1,técnica e preço,search log,técnica e preço - Licitação de técnica e preço,"técnica e preço - Licitação de técnica e preço, Proposta técnica","técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço de mercado","técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço de mercado, Proposta de preço","técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço de mercado, Proposta de preço, Preço unitário","técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço de mercado, Proposta de preço, Preço unitário, Proposta","técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço de mercado, Proposta de preço, Preço unitário, Proposta, Bens e serviços de informática","técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço de mercado, Proposta de preço, Preço unitário, Proposta, Bens e serviços de informática, Qualificação técnica","técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço de mercado, Proposta de preço, Preço unitário, Proposta, Bens e serviços de informática, Qualificação técnica, Documento téc...","técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço de mercado, Proposta de preço, Preço unitário, Proposta, Bens e serviços de informática, Qualificação técnica, Documento téc..."
1,2,restos a pagar,search log,restos a pagar - Recolhimento,"restos a pagar - Recolhimento, Exercício financeiro anterior","restos a pagar - Recolhimento, Exercício financeiro anterior, Passivo circulante","restos a pagar - Recolhimento, Exercício financeiro anterior, Passivo circulante, Juros","restos a pagar - Recolhimento, Exercício financeiro anterior, Passivo circulante, Juros, Cota-parte","restos a pagar - Recolhimento, Exercício financeiro anterior, Passivo circulante, Juros, Cota-parte, Despesas de exercícios anteriores","restos a pagar - Recolhimento, Exercício financeiro anterior, Passivo circulante, Juros, Cota-parte, Despesas de exercícios anteriores, Obrigação","restos a pagar - Recolhimento, Exercício financeiro anterior, Passivo circulante, Juros, Cota-parte, Despesas de exercícios anteriores, Obrigação, Crime contra as finanças públicas","restos a pagar - Recolhimento, Exercício financeiro anterior, Passivo circulante, Juros, Cota-parte, Despesas de exercícios anteriores, Obrigação, Crime contra as finanças públicas, Quitação","restos a pagar - Recolhimento, Exercício financeiro anterior, Passivo circulante, Juros, Cota-parte, Despesas de exercícios anteriores, Obrigação, Crime contra as finanças públicas, Quitação"
2,3,aditivo a contrato,search log,aditivo a contrato - Termo aditivo,"aditivo a contrato - Termo aditivo, Aditivo qualitativo","aditivo a contrato - Termo aditivo, Aditivo qualitativo, Acréscimo","aditivo a contrato - Termo aditivo, Aditivo qualitativo, Acréscimo, Contrato administrativo","aditivo a contrato - Termo aditivo, Aditivo qualitativo, Acréscimo, Contrato administrativo, Aditivo quantitativo","aditivo a contrato - Termo aditivo, Aditivo qualitativo, Acréscimo, Contrato administrativo, Aditivo quantitativo, Formalização do contrato","aditivo a contrato - Termo aditivo, Aditivo qualitativo, Acréscimo, Contrato administrativo, Aditivo quantitativo, Formalização do contrato, Reformulação","aditivo a contrato - Termo aditivo, Aditivo qualitativo, Acréscimo, Contrato administrativo, Aditivo quantitativo, Formalização do contrato, Reformulação, Atualização","aditivo a contrato - Termo aditivo, Aditivo qualitativo, Acréscimo, Contrato administrativo, Aditivo quantitativo, Formalização do contrato, Reformulação, Atualização","aditivo a contrato - Termo aditivo, Aditivo qualitativo, Acréscimo, Contrato administrativo, Aditivo quantitativo, Formalização do contrato, Reformulação, Atualização"
3,4,adesão a ata de registro de preços,search log,adesão a ata de registro de preços - Sistema de Registro de Preços,"adesão a ata de registro de preços - Sistema de 

In [ ]:
result = df_query.drop(['ID','SOURCE'],axis=1).head(2).T

In [ ]:
result

,0,1
TEXT,técnica e preço,restos a pagar
1,técnica e preço - Licitação de técnica e preço,restos a pagar - Recolhimento
2,"técnica e preço - Licitação de técnica e preço, Proposta técnica","restos a pagar - Recolhimento, Exercício financeiro anterior"
3,"técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço de mercado","restos a pagar - Recolhimento, Exercício financeiro anterior, Passivo circulante"
4,"técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço de mercado, Proposta de preço","restos a pagar - Recolhimento, Exercício financeiro anterior, Passivo circulante, Juros"
5,"técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço de mercado, Proposta de preço, Preço unitário","restos a pagar - Recolhimento, Exercício financeiro anterior, Passivo circulante, Juros, Cota-parte"
6,"técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço de mercado, Proposta de preço, Preço unitário, Proposta","restos a pagar - Recolhimento, Exercício financeiro anterior, Passivo circulante, Juros, Cota-parte, Despesas de exercícios anteriores"
7,"técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço de mercado, Proposta de preço, Preço unitário, Proposta, Bens e serviços de informática","restos a pagar - Recolhimento, Exercício financeiro anterior, Passivo circulante, Juros, Cota-parte, Despesas de exercícios anteriores, Obrigação"
8,"técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço de mercado, Proposta de preço, Preço unitário, Proposta, Bens e serviços de informática, Qualificação técnica","restos a pagar - Recolhimento, Exercício financeiro anterior, Passivo circulante, Juros, Cota-parte, Despesas de exercícios anteriores, Obrigação, Crime contra as finanças públicas"
9,"técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço de mercado, Proposta de preço, Preço unitário, Proposta, Bens e serviços de informática, Qualificação técnica, Documento téc...","restos a pagar - Recolhimento, Exercício financeiro anterior, Passivo circulante, Juros, Cota-parte, Despesas de exercícios anteriores, Obrigação, Crime contra as finanças públicas, Quitação"


In [ ]:
result.to_excel("../docs/table/table_query_expansion_example.xlsx", index=False)